#Lab 1 - Programação - Integral Regra do Trapézio

## Código fonte sequencial:

In [15]:
%%writefile itegral_omp.c


#include <stdio.h>
#include <math.h>
#include <time.h>



double f(double x) {
    return sin(x);
}


double trapezoidal_rule(double a, double b, int n) {
    double h = (b - a) / n;
    double approx = (f(a) + f(b)) / 2.0;


    for (int i = 1; i <= n - 1; i++) {
        double x_i = a + i * h;
        approx += f(x_i);
    }


    approx = h * approx;
    return approx;
}


int main() {
    double a = 0.0, b = M_PI;
    int n = 100000000;


    clock_t start_time = clock();


    double result = trapezoidal_rule(a, b, n);


    clock_t end_time = clock();
    double execution_time = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;


    printf("Resultado: %f\n", result);
    printf("Tempo de execução (sequencial): %f segundos\n", execution_time);


    return 0;
}


Overwriting itegral_omp.c


In [16]:
!gcc -fopenmp itegral_omp.c -lm -o itegral_omp
!./itegral_omp

Resultado: 2.000000
Tempo de execução (sequencial): 2.109870 segundos


##Código fonte implementação paralela e melhorias:

In [6]:
%%writefile itegral_paralel_omp.c


#include <stdio.h>
#include <math.h>
#include <omp.h>


double f(double x) {
    return sin(x);
}


double trapezoidal_rule(double a, double b, int n) {
    double h = (b - a) / n;
    double approx = (f(a) + f(b)) / 2.0;


    #pragma omp parallel for reduction(+:approx)
    for (int i = 1; i <= n - 1; i++) {
        double x_i = a + i * h;
        approx += f(x_i);
    }


    approx = h * approx;
    return approx;
}


int main() {
    double a = 0.0, b = M_PI;
    int n = 100000000;


    double start_time = omp_get_wtime();


    double result = trapezoidal_rule(a, b, n);


    double end_time = omp_get_wtime();
    double execution_time = end_time - start_time;


    printf("Resultado: %f\n", result);
    printf("Tempo de execução (OpenMP): %f segundos\n", execution_time);


    return 0;
}


Overwriting itegral_paralel_omp.c


In [7]:
!gcc -fopenmp itegral_paralel_omp.c -lm -o itegral_paralel_omp
!./itegral_paralel_omp

Resultado: 2.000000
Tempo de execução (OpenMP): 1.584109 segundos


#Lab 2 - Avaliação de desempenho


##Código Fonte Critical:

In [17]:
%%writefile somatorio_crit_openmb.c


#include <stdio.h>
#include <omp.h>
#include <stdlib.h>
#include <time.h>


#define SIZE (1 << 30)
#define REPS 10


// Preenchendo o vetor somente com 1.
void fill_array(int *arr, long long size) {
    for (long long i = 0; i < size; i++) {
        arr[i] = 1;
    }
}


double sum_with_critical(int *arr, long long size, int threads) {
    long long sum = 0;
    double start_time, end_time;


    start_time = omp_get_wtime();


    #pragma omp parallel num_threads(threads)
    {
        long long partial_sum = 0;
        #pragma omp for
        for (long long i = 0; i < size; i++) {
            partial_sum += arr[i];
        }
        #pragma omp critical
        sum += partial_sum;
    }
    end_time = omp_get_wtime();
    return end_time - start_time;
}


void run_experiment(int *arr, long long size) {
    int threads[] = {1, 2, 3, 4, 5, 6};
    double time_critical[6] = {0};


    // Realizando 10 execuções para média
    for (int r = 0; r < REPS; r++) {
        for (int i = 0; i < 6; i++) {
            time_critical[i] += sum_with_critical(arr, size, threads[i]);
        }
    }


    // Calculando a média
    for (int i = 0; i < 6; i++) {
        time_critical[i] /= REPS;
    }


    // Imprimindo a tabela
    printf("Threads\tCritical\n");
    for (int i = 0; i < 6; i++) {
        printf("%d\t%f\n", threads[i], time_critical[i]);
    }
}


int main() {
    // Alocando o vetor dinamicamente para suportar 2^30 elementos
    int *arr = (int*) malloc(SIZE * sizeof(int));


    if (arr == NULL) {
        printf("Erro ao alocar o vetor\n");
        return 1;
    }


    fill_array(arr, SIZE);


    run_experiment(arr, SIZE);


    // Liberando a memória alocada
    free(arr);


    return 0;
}


Writing somatorio_crit_openmb.c


In [18]:
!gcc -fopenmp somatorio_crit_openmb.c -o somatorio_crit_openmb
!./somatorio_crit_openmb

Threads	Critical
1	3.124607
2	1.964076
3	2.146551
4	2.386907
5	2.410768
6	2.409652


##Código Fonte Reduction:


In [19]:
%%writefile somatorio_reduc_openmb.c


#include <stdio.h>
#include <omp.h>
#include <stdlib.h>
#include <time.h>


#define SIZE (1 << 30)
#define REPS 10


// Preenchendo o vetor somente com 1.
void fill_array(int *arr, long long size) {
    for (long long i = 0; i < size; i++) {
        arr[i] = 1;
    }
}


double sum_with_reduction(int *arr, long long size, int threads) {
    long long sum = 0;
    double start_time, end_time;


    start_time = omp_get_wtime();


    #pragma omp parallel for num_threads(threads) reduction(+:sum)
    for (long long i = 0; i < size; i++) {
        sum += arr[i];
    }


    end_time = omp_get_wtime();


    return end_time - start_time;
}


void run_experiment(int *arr, long long size) {
    int threads[] = {1, 2, 3, 4, 5, 6};
    double time_reduction[6] = {0};


    // Realizando 10 execuções para média
    for (int r = 0; r < REPS; r++) {
        for (int i = 0; i < 6; i++) {
            time_reduction[i] += sum_with_reduction(arr, size, threads[i]);
        }
    }


    // Calculando a média
    for (int i = 0; i < 6; i++) {
        time_reduction[i] /= REPS;
    }


    // Imprimindo a tabela
    printf("Threads\tReduction\n");
    for (int i = 0; i < 6; i++) {
        printf("%d\t%f\n", threads[i], time_reduction[i]);
    }
}


int main() {
    // Alocando o vetor dinamicamente para suportar 2^30 elementos
    int *arr = (int*) malloc(SIZE * sizeof(int));


    if (arr == NULL) {
        printf("Erro ao alocar o vetor\n");
        return 1;
    }
    fill_array(arr, SIZE);
    run_experiment(arr, SIZE);


    // Liberando a memória alocada
    free(arr);


    return 0;
}


Writing somatorio_reduc_openmb.c


In [20]:
!gcc -fopenmp somatorio_reduc_openmb.c -o somatorio_reduc_openmb
!./somatorio_reduc_openmb

Threads	Reduction
1	2.994960
2	2.199437
3	2.172225
4	2.118183
5	2.302955
6	2.436342


#Lab 3 - Nova multiplicação de matrizes

##Código fonte e execução multiplicação de matriz linear :

In [34]:
%%writefile mult_sequencial.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void multiply_matrices(int **A, int **B, int **C, int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = 0;
            for (int k = 0; k < N; k++) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }
}

int main() {
    int N = 1000; // Tamanho da matriz NxN
    int **A, **B, **C;

    // Alocação dinâmica das matrizes
    A = (int **)malloc(N * sizeof(int *));
    B = (int **)malloc(N * sizeof(int *));
    C = (int **)malloc(N * sizeof(int *));
    for (int i = 0; i < N; i++) {
        A[i] = (int *)malloc(N * sizeof(int));
        B[i] = (int *)malloc(N * sizeof(int));
        C[i] = (int *)malloc(N * sizeof(int));
    }

    // Inicialização das matrizes A e B com valores aleatórios
    srand(time(NULL));
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = rand() % 100;
            B[i][j] = rand() % 100;
        }
    }

    // Medição do tempo de execução
    clock_t start = clock();
    multiply_matrices(A, B, C, N);
    clock_t end = clock();

    double time_taken = ((double)(end - start)) / CLOCKS_PER_SEC;
    printf("Tempo de execução (Sequencial): %f segundos\n", time_taken);

    // Liberação da memória alocada
    for (int i = 0; i < N; i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
    }
    free(A);
    free(B);
    free(C);

    return 0;
}

Overwriting mult_sequencial.c


In [35]:
!gcc -o mult_sequencial mult_sequencial.c -fopenmp
!./mult_sequencial

Tempo de execução (Sequencial): 13.884984 segundos


##Código fonte e execução multiplicação  de matrizes OpenMP.


In [36]:
%%writefile mult_mp.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

void multiply_matrices(int **A, int **B, int **C, int N) {
    #pragma omp parallel for
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = 0;
            for (int k = 0; k < N; k++) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }
}

int main() {
    int N = 1000; // Tamanho da matriz NxN
    int **A, **B, **C;

    // Alocação dinâmica das matrizes
    A = (int **)malloc(N * sizeof(int *));
    B = (int **)malloc(N * sizeof(int *));
    C = (int **)malloc(N * sizeof(int *));
    for (int i = 0; i < N; i++) {
        A[i] = (int *)malloc(N * sizeof(int));
        B[i] = (int *)malloc(N * sizeof(int));
        C[i] = (int *)malloc(N * sizeof(int));
    }

    // Inicialização das matrizes A e B com valores aleatórios
    srand(time(NULL));
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = rand() % 100;
            B[i][j] = rand() % 100;
        }
    }

    int num_threads[] = {1, 2, 3, 4, 5, 6};
    double time_taken[6];

    for (int t = 0; t < 6; t++) {
        omp_set_num_threads(num_threads[t]);
        double start = omp_get_wtime();

        multiply_matrices(A, B, C, N);

        double end = omp_get_wtime();
        time_taken[t] = end - start;

        printf("Tempo de execução com %d threads: %f segundos\n", num_threads[t], time_taken[t]);
    }

    // Liberação da memória alocada
    for (int i = 0; i < N; i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
    }
    free(A);
    free(B);
    free(C);

    return 0;
}


Writing mult_mp.c


In [37]:
!gcc -o mult_mp mult_mp.c -fopenmp
!./mult_mp

Tempo de execução com 1 threads: 14.003931 segundos
Tempo de execução com 2 threads: 14.836055 segundos
Tempo de execução com 3 threads: 14.402459 segundos
Tempo de execução com 4 threads: 14.110781 segundos
Tempo de execução com 5 threads: 14.404463 segundos
Tempo de execução com 6 threads: 14.478287 segundos
